
---
---

# 1. Setup

---
---

### Import Packages

##### Import the general packages

In [ ]:
from google.colab import drive
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter

##### Install & import the tsfresh libraries

In [ ]:
!pip install tsfresh

     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 665kB 12.2MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
     |████████████████████████████████| 102kB 8.8MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=a4fd4f64c8f94088302e2c75e974bfbda5af65924f47d8ad83a0841c9585a833
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=be317318389cf691c3959cc73dbeebeee6f11a08d75e26d1e82b38d91e4f977c
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built contextvars locket
ERROR: tensorflow-probability 0.11.0 has requirement cloudpickle==1.3, but you'll have cloudpickle 1.6.0 which is incompatible.
  Found existing installation: cloudpickle 1.3.0
    Uninstalling cloudpickle-1.3.0:
      Su

In [ ]:
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##### Import and setup the Json package for loading in and storing data

In [ ]:
import json

class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_json'):
            return obj.to_json(orient='records')
        return json.JSONEncoder.default(self, obj)

### Set the maximum number of rows and column in the outputs

In [ ]:
pd.set_option('max_rows', 25)

### Mount the drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set the directory for reading the data

In [ ]:
os.chdir('/content/drive/My Drive/4th Year/CA4015/assignment2/Submission/data')    # Ivan's directory
# os.chdir('/content/drive/My Drive/assignment2/Submission/data')                     # Nathan's directory

---
---

# 2. Read in the data

---
---

### Read in the dictionary data from the Json file

In [ ]:
json_df_data = json.load(open('map_of_subject_id_to_its_cleaned_sensor_data.json'))
# for the no step data
# json_df_data = json.load(open('no_step_map_of_subject_id_to_its_cleaned_sensor_data.json'))

In [ ]:
id_to_df_map = {}
for subject_id, sensor_df in tqdm(json_df_data.items()):
    id_to_df_map[subject_id] = pd.read_json(json_df_data[subject_id])

### Manually inspect the read in data

In [ ]:
i = 0
for student_id, df in id_to_df_map.items():
    print("--------------------", student_id, "--------------------")
    print(df)

    # only output 3 subjects and their sensor data
    if i == 2:
        break
    i = i + 1

-------------------- 46343 --------------------
       second    x_move    y_move    z_move  heart_rate  step_count  psg_status
0         390 -0.540527  0.680496 -0.271774        90.0   79.820000           0
1         391 -0.426881  0.920799 -0.093921        90.0  109.617059           0
2         392 -0.448256  0.816442 -0.228871        90.0  125.870000           0
3         393 -0.464479  0.765035 -0.432477        90.0  125.870000           0
4         394 -0.494360  0.798880 -0.076882        90.0  125.870000           0
...       ...       ...       ...       ...         ...         ...         ...
16556   16946 -0.441763 -0.525671  0.723509        73.0    0.000000           0
16557   16947 -0.441267 -0.525272  0.724310        73.0    0.000000           0
16558   16948 -0.441876 -0.525352  0.724083        73.0    0.000000           0
16559   16949 -0.442227 -0.525543  0.723603        73.0    0.000000           0
16560   16950 -0.441905 -0.525534  0.723882        73.0    0.000000     

---
---

# 3. Get the data into the correct format for extracting the features

---
---

### Add an 'id' column to each row specifying which bin of seconds each row falls in

In [ ]:
bin_size = 30

In [ ]:
map_subject_to_df_with_id = {}
for subject_id, fixed_sensor_df in tqdm(id_to_df_map.items()):

    print("---------------", subject_id, "-----------------")

    print(fixed_sensor_df.shape)
  	# dropna's
    no_nans_fixed_sensor_df = fixed_sensor_df.dropna()
    print(no_nans_fixed_sensor_df.shape)

    # get the value of the maximum second in this dataframe
    max_second_in_df = int(round(max(no_nans_fixed_sensor_df.second) + 0.5))

    # create a new dataframe that we will populate
    new_df = pd.DataFrame(columns=(list(no_nans_fixed_sensor_df.columns).extend(["session_id"])))

    session_number = 0
    # iterate through each second interval in this dataframe
    for i in np.arange(0, max_second_in_df + bin_size, bin_size):

        # get the rows between second "i - 1" and second "i"
        rows_in_session_df = pd.DataFrame(no_nans_fixed_sensor_df.loc[(no_nans_fixed_sensor_df.second >= (i)) & (no_nans_fixed_sensor_df.second < i + bin_size)])
        
        if not rows_in_session_df.empty:
            # assign the session_id label to this row
            rows_in_session_df['session_id'] = session_number

            # join these rows to the rest of the rows
            new_df = pd.concat([new_df, rows_in_session_df], axis=0)

            session_number += 1

    map_subject_to_df_with_id[subject_id] = new_df

--------------- 46343 -----------------
(16561, 7)
(16561, 7)
--------------- 759667 -----------------
(14184, 7)
(14184, 7)
--------------- 781756 -----------------
(29369, 7)
(29369, 7)
--------------- 844359 -----------------
(26881, 7)
(26881, 7)
--------------- 1066528 -----------------
(28389, 7)
(28389, 7)
--------------- 1360686 -----------------
(27695, 7)
(27695, 7)
--------------- 1449548 -----------------
(28561, 7)
(28561, 7)
--------------- 1455390 -----------------
(28621, 7)
(28621, 7)
--------------- 1818471 -----------------
(28711, 7)
(28711, 7)
--------------- 2598705 -----------------
(28591, 7)
(28591, 7)
--------------- 2638030 -----------------
(28411, 7)
(28411, 7)
--------------- 3509524 -----------------
(12448, 7)
(12448, 7)
--------------- 3997827 -----------------
(28711, 7)
(28711, 7)
--------------- 4018081 -----------------
(14940, 7)
(14940, 7)
--------------- 4314139 -----------------
(28801, 7)
(28801, 7)
--------------- 4426783 -----------------
(29

##### Have a look at the values in the 'id' column

In [ ]:
for sub_id, df in map_subject_to_df_with_id.items():
    seen = {}
    for v in df.session_id:

        if v in seen:
            seen[v] += 1
            
        else:
            seen[v] = 1
            
    print(seen)

{0: 30, 1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30, 7: 30, 8: 30, 9: 30, 10: 30, 11: 30, 12: 30, 13: 30, 14: 30, 15: 30, 16: 30, 17: 30, 18: 30, 19: 30, 20: 30, 21: 30, 22: 30, 23: 30, 24: 30, 25: 30, 26: 30, 27: 30, 28: 30, 29: 30, 30: 30, 31: 30, 32: 30, 33: 30, 34: 30, 35: 30, 36: 30, 37: 30, 38: 30, 39: 30, 40: 30, 41: 30, 42: 30, 43: 30, 44: 30, 45: 30, 46: 30, 47: 30, 48: 30, 49: 30, 50: 30, 51: 30, 52: 30, 53: 30, 54: 30, 55: 30, 56: 30, 57: 30, 58: 30, 59: 30, 60: 30, 61: 30, 62: 30, 63: 30, 64: 30, 65: 30, 66: 30, 67: 30, 68: 30, 69: 30, 70: 30, 71: 30, 72: 30, 73: 30, 74: 30, 75: 30, 76: 30, 77: 30, 78: 30, 79: 30, 80: 30, 81: 30, 82: 30, 83: 30, 84: 30, 85: 30, 86: 30, 87: 30, 88: 30, 89: 30, 90: 30, 91: 30, 92: 30, 93: 30, 94: 30, 95: 30, 96: 30, 97: 30, 98: 30, 99: 30, 100: 30, 101: 30, 102: 30, 103: 30, 104: 30, 105: 30, 106: 30, 107: 30, 108: 30, 109: 30, 110: 30, 111: 30, 112: 30, 113: 30, 114: 30, 115: 30, 116: 30, 117: 30, 118: 30, 119: 30, 120: 30, 121: 30, 122: 30, 12

---
---

# 4. Create a map of 'session_id' to 'psg_status' for each student_id

---
---

### For each subject, create a map between each of their session_ids and that sessions psg_status

In [ ]:
map_subject_id_to_a_map_of_the_session_id_to_psg_status = {}

for subject_id, sensor_df in tqdm(map_subject_to_df_with_id.items()):

    # for this subject, create a dictionary to map their sessions to their psg status'
    subjects_session_to_psg_map = {}

    for session_id in list(set(sensor_df.session_id)):
    
        # get all id entries in df where psg_status = sleep_state
        all_psg_status = sensor_df[sensor_df['session_id'] == session_id]['psg_status']

        # get the most common psg_status across all rows with this session_id
        most_common_psg_status = Counter(all_psg_status).most_common(1)[0][0]

        # create an entry in the subject dictionary of a map between the session_id and the most common psg_status
        subjects_session_to_psg_map[session_id] = most_common_psg_status

    # add this subjects dictionaries to the map of each subject_id to their dictionaries
    map_subject_id_to_a_map_of_the_session_id_to_psg_status[subject_id] = subjects_session_to_psg_map

##### Manually inspect this dictionary

In [ ]:
for sub_id, id_to_psg_dict in map_subject_id_to_a_map_of_the_session_id_to_psg_status.items():

    # print the subject_id
    print("----------------", sub_id, "---------------------")
    
    # iterate through the dictionary and print the values
    for session_id, psg_status in id_to_psg_dict.items():
        print(session_id, "->", psg_status)

    # stop after one subject_id
    break

---------------- 46343 ---------------------
0 -> 0
1 -> 0
2 -> 0
3 -> 0
4 -> 0
5 -> 0
6 -> 0
7 -> 0
8 -> 0
9 -> 0
10 -> 0
11 -> 0
12 -> 0
13 -> 0
14 -> 0
15 -> 0
16 -> 0
17 -> 0
18 -> 0
19 -> 0
20 -> 0
21 -> 0
22 -> 0
23 -> 0
24 -> 0
25 -> 0
26 -> 0
27 -> 0
28 -> 0
29 -> 0
30 -> 0
31 -> 0
32 -> 0
33 -> 0
34 -> 1
35 -> 1
36 -> 1
37 -> 1
38 -> 2
39 -> 2
40 -> 2
41 -> 2
42 -> 2
43 -> 2
44 -> 2
45 -> 2
46 -> 2
47 -> 2
48 -> 2
49 -> 2
50 -> 2
51 -> 2
52 -> 2
53 -> 2
54 -> 2
55 -> 2
56 -> 3
57 -> 3
58 -> 3
59 -> 3
60 -> 3
61 -> 3
62 -> 3
63 -> 3
64 -> 3
65 -> 3
66 -> 3
67 -> 3
68 -> 3
69 -> 3
70 -> 3
71 -> 3
72 -> 3
73 -> 3
74 -> 3
75 -> 3
76 -> 3
77 -> 3
78 -> 3
79 -> 3
80 -> 3
81 -> 3
82 -> 3
83 -> 3
84 -> 3
85 -> 3
86 -> 3
87 -> 3
88 -> 3
89 -> 3
90 -> 3
91 -> 3
92 -> 3
93 -> 3
94 -> 3
95 -> 3
96 -> 3
97 -> 3
98 -> 3
99 -> 3
100 -> 3
101 -> 3
102 -> 3
103 -> 3
104 -> 3
105 -> 3
106 -> 3
107 -> 3
108 -> 3
109 -> 3
110 -> 0
111 -> 0
112 -> 0
113 -> 0
114 -> 1
115 -> 2
116 -> 2
117 -> 2
118

---
---

# 5. Extract the features 

---
---

### Extract the features for each sensor

In [ ]:
map_id_to_extracted_features = {}
for subject_id, cleaned_sensor_df in tqdm(map_subject_to_df_with_id.items()):

    print("---------------------------------------------------")
    print("======================", subject_id, "======================")
    print("---------------------------------------------------")

    no_psg_status_cleaned_df = cleaned_sensor_df.drop(columns=["psg_status"], axis=1).dropna()

    no_psg_status_cleaned_df["session_id"] = no_psg_status_cleaned_df["session_id"].astype(str)

    extracted_features = extract_features(no_psg_status_cleaned_df, column_value=None, column_sort="second", column_id="session_id")

    print(extracted_features.shape)
    extracted_features = extracted_features.dropna(axis='columns')
    
    print(extracted_features.shape)

    map_id_to_extracted_features[subject_id] = extracted_features

---------------------------------------------------
====================== 46343 ======================
---------------------------------------------------


Feature Extraction: 100%|██████████| 5/5 [02:36<00:00, 31.23s/it]


(553, 3895)
(553, 1058)
---------------------------------------------------
====================== 759667 ======================
---------------------------------------------------


Feature Extraction: 100%|██████████| 5/5 [02:12<00:00, 26.44s/it]


(474, 3895)
(474, 1058)
---------------------------------------------------
====================== 781756 ======================
---------------------------------------------------


...

(960, 3895)
(960, 1058)
---------------------------------------------------
====================== 9618981 ======================
---------------------------------------------------


Feature Extraction: 100%|██████████| 5/5 [04:23<00:00, 52.64s/it]


(944, 3895)
(944, 1058)
---------------------------------------------------
====================== 9961348 ======================
---------------------------------------------------


Feature Extraction: 100%|██████████| 5/5 [03:21<00:00, 40.22s/it]


(719, 3895)
(719, 1058)



### Store the extracted features in a dictionary

In [ ]:
# put the dataframe in the json file
with open('map_subject_id_to_its_unfiltered_extracted_features_df.json', 'w') as fp:
    json.dump(map_id_to_extracted_features, fp, cls=JSONEncoder)

In [ ]:
# for the no step data
# with open('no_step_map_subject_id_to_its_unfiltered_extracted_features_df.json', 'w') as fp:
#    json.dump(map_id_to_extracted_features, fp, cls=JSONEncoder)

---
---

# 6. Select the most relevant features from all of these extracted features

---
---

In [ ]:
map_id_to_filtered_extracted_features = {}
for subject_id, extracted_features_df in tqdm(map_id_to_extracted_features.items()):

    print("---------------------------------------------------")
    print("======================", subject_id, "======================")
    print("---------------------------------------------------")
    
    map_of_session_id_to_psg_status = map_subject_id_to_a_map_of_the_session_id_to_psg_status[subject_id]

    target_array = np.array(list(map_of_session_id_to_psg_status.values()))

    print(extracted_features_df.shape)
    features_filtered = select_features(extracted_features_df, target_array)
    print(features_filtered.shape)

    map_id_to_filtered_extracted_features[subject_id] = features_filtered

---------------------------------------------------
====================== 46343 ======================
---------------------------------------------------
(553, 1058)
(553, 357)
---------------------------------------------------
====================== 759667 ======================
---------------------------------------------------
(474, 1058)
(474, 385)
---------------------------------------------------
====================== 781756 ======================
---------------------------------------------------
(980, 1058)
(980, 270)
---------------------------------------------------
====================== 844359 ======================
---------------------------------------------------
(897, 1058)
(897, 362)
---------------------------------------------------
====================== 1066528 ======================
---------------------------------------------------
(947, 1755)
(947, 653)
---------------------------------------------------
====================== 1360686 =================

---
---

# 7. Store this data in a json file

---
---

### Store the extracted features dictionary in a Json file

In [ ]:
# put the dataframe in the json file
with open('map_subject_id_to_its_filtered_extracted_features_df.json', 'w') as fp:
    json.dump(map_id_to_filtered_extracted_features, fp, cls=JSONEncoder)
# for the no step data
# with open('no_step_map_subject_id_to_its_filtered_extracted_features_df.json', 'w') as fp:
#     json.dump(map_id_to_filtered_extracted_features, fp, cls=JSONEncoder)

### Store the psg_status map to all of the sessions with that status in a Json file

In [ ]:
# put the dataframe in the json file
with open('map_subject_id_to_a_map_of_the_session_id_to_psg_status.json', 'w') as fp:
    json.dump(map_subject_id_to_a_map_of_the_session_id_to_psg_status, fp, cls=JSONEncoder)